# Microtask-2

Same as microtask-1, but now using pandas.

> Produce a notebook showing (and producing) a list with the activity per quarter: number of new committers, submitters of issues, and submitters of pull/merge requests, number of items (commits, issues, pull/merge requests), number of repositories with new items (all of this per quarter) as a table and as a CSV file using plain python3 (using pandas).


I am using the same data source file which is used in the [microtask-0](https://github.com/vchrombie/chaoss-microtasks/blob/master/microtask-0/microtask-0.ipynb) i.e, [elasticsearch-py](https://github.com/elastic/elasticsearch-py) project which is located in the `data/` folder of the repository.

## Importing the neccessary modules

In [1]:
# json library is used to handle json files, here, it is the data source retrieved by the perceval module.
import json 
# to write and read csv files, to show the output in the end
import csv  

# importing pandas
# pandas is used for handling huge data using dataframe
import pandas as pd

# to handle the time formats, like to determine 'created_at' of an issue or pr.
import datetime  
# dictionaries are a convenient way to store data for later retrieval by name (key).
from collections import defaultdict  

# it is used to send http requests, I used to get the year in which the project created to do the analysis, using requests and github api.
import requests 

## Defining Quaters

The calendar year can be divided into four quarters, often abbreviated as Q1, Q2, Q3, and Q4.
- First quarter, Q1: 1 January – 31 March (90 days or 91 days in leap years)
- Second quarter, Q2: 1 April – 30 June (91 days)
- Third quarter, Q3: 1 July – 30 September (92 days)
- Fourth quarter, Q4: 1 October – 31 December (92 days)

Reference: https://en.wikipedia.org/wiki/Calendar_year

Each Quater is represented as **Qi yyyy** where *i* is the quater number and _yyyy_ is the year.

In [2]:
# as said the year can be divided into four different quaters 
# so I divided the year into 4 quaters according to the information mentioned above.
QUARTERS = (
    ({'month':1,'day':1},  {'month':3,'day':31}),
    ({'month':4,'day':1},  {'month':6,'day':30}),
    ({'month':7,'day':1},  {'month':9,'day':30}),
    ({'month':10,'day':1}, {'month':12,'day':31}),
)


# there are three types of contributions - commit, issue, and pr
# so, I created a tuple which has the contribution types
ctypes = ('commit','pull_request','issue')

## Functions to return the details of the contribution types

_Commit_ has a different json structure when compared to *issue* and *pull_request*.

In [3]:
# function to get the required details of commits
# commit has a different json structure unlike issue/pr

def details_commit(commit):
    # load the commit data into the object
    data = commit['data']
    # traverse through the json line to find the required data
    content ={
            # get the hash of the commit
            'hash': data['commit'],
            # get the author_name
            'author': data['Author'],  
            # get the date at which the commit was created
            'created_date': datetime.datetime.strptime(data['CommitDate'],"%a %b %d %H:%M:%S %Y %z")  
    }
    return content

In [4]:
# function to get the required details of issue/pull requests
# as issue/pr has the same json structure in the data source scraped by perceval
# I wrote a single function to get the either issue/pr details 

def details_ipr(item):
    # load the commit data into the object
    data = item['data']
    # traverse through the json line to find the required data
    content ={
            # get the hash of the issue/pr
            'hash': data['id'],
            # get the author_name
            'author': data['user']['login'],  
            # get the date at which the issue/pr was created
            'created_date': datetime.datetime.strptime(data['created_at'],"%Y-%m-%dT%H:%M:%SZ")  
    }
    return content 

## Dividing the data source into contribution types

In [5]:
# intializing the content dataframe with the required details
content_df = {
    # commit has the column >> hash, author, date
    'commit':pd.DataFrame(columns=['hash', 'author', 'created_date']),
    # issue has the column >> hash, author, date
    'issue':pd.DataFrame(columns=['hash', 'author', 'created_date']),
    # pr has the column >> hash, author, date
    'pull_request':pd.DataFrame(columns=['hash', 'author', 'created_date'])
}

# contents is to store the details of each contribution whether it is a commit, issue or pr.
# using a defaultdict of list so that I can store the sorted details according to the ctype as (key, value) 
contents = defaultdict(list)

# to filter out commit, issue, pr details from the data source and store them seperately in dict.
# loading the file into an object
with open('../data/elasticsearch-py.json') as datasrc:
    for line in datasrc:
        # load the line in the json format so as to iterate to get the required results
        line = json.loads(line)
        # if it is a commit, get the details of commit
        if line['category'] == 'commit':    
            content = details_commit(line) 
        # if it is a issue, get the details of issue
        elif line['category'] == 'issue':    
            content = details_ipr(line)
        # if it is a pr, get the details of pr
        elif line['category'] == 'pull_request':    
            content = details_ipr(line) 
        # add the (key, value) to the list
        contents[line['category']].append(content)
        
for item in contents:
    content_df[item]=content_df[item].append(contents[item],sort=False)

## Splitting the data quaterly

In [6]:
# this cell is to get the year in which the project was actually created and get the current year to iterate for splitting the quaters

# using github api and requests, the data of the repository is stored in the object
repodata =json.loads(requests.get("https://api.github.com/repos/elastic/elasticsearch-py").text)

# using datetime library, got the year in which the project was created and the current year
created =datetime.datetime.strptime(repodata['created_at'][:10], "%Y-%m-%d").year
present =datetime.datetime.strptime(repodata['updated_at'][:10], "%Y-%m-%d").year

In [7]:
# funtion to divide the contributions into quaterly based on the QUATERS

# the first_year and last_year is divided into for quaters
def quarterwise(first_year,last_year):
    # iterating through the years first
    for year in range(first_year, last_year+1):
        # iterating through the enumerated quaters
        for quarter,(start,end) in enumerate(QUARTERS):
            # returning the quaters start and end time period
            start = datetime.datetime(year,**start)
            end = datetime.datetime(year,**end)
            # yield is used to return multiple files in one particular order
            yield year,quarter,start,end  

In [8]:
# initializing a empty quaters list to store the quaters of the project
quarters = []

# using again a defaultdict(list) to store the activites and it's vaues as quaters.
activities = defaultdict(list)

# newcontributors as list in order to append each time a new contributor arrives
newcontributors = defaultdict(list)
# oldcontributors as set so that dupplicated can't crawl into the set
oldcontributors = defaultdict(set)

# generating the quaters from `created`  year to `present`  year which are scraped earlier
for year,quarter,start,end in quarterwise(created,present):
    # add `Qi yyyy`  format as a quater in the quaters list
    quarters.append(r"Q%d %d"%(quarter+1,year))
    # iterating through the contribution types in order to segregate their values into the dict
    for ctype in ctypes:
        # copying the specific contribution type dataframe 
        df = content_df[ctype]
        # removing the timzone as we don't need it
        df['created_date']= df['created_date'].apply(lambda x:x.replace(tzinfo=None))
        # checking if the date of contribtion (commit/issue/pr) created is in between start & end
        df = df[(start<=df['created_date'])& (df['created_date'] <=end)]
        # filtering out the contributors using set -> to remove duplicates
        contributors = set(df['author'])
        # filtering out the new contributors -> total-old
        newcontributors[ctype].append(len(contributors-oldcontributors[ctype]))
        # updating the current contributors to the old contributors
        oldcontributors[ctype].update(contributors)
        # filtering out the activities using hash ids (unique) of the commit/issue/pr
        activities[ctype].append(df['hash'].nunique())

## Showing the Activity

In [9]:
# to print the total activity 
print("Quaterwise Total Activity\n")
for item in dict(activities):
    # print the total activity quaterly
    print (item, dict(activities)[item])  

# small hack to produce a new line to make space. 
#just for the funcs to look symmetric while printing :P
print() 

# to print the new activity 
print("Quaterwise New Contributors Activity\n")
# iterating through the newcontributors dict 
for item in dict(newcontributors):
    # print the new activity quaterly
    print (item, dict(activities)[item])  

Quaterwise Total Activity

commit [0, 78, 154, 94, 126, 33, 30, 38, 45, 37, 27, 80, 28, 29, 21, 34, 27, 28, 31, 57, 44, 26, 10, 37, 4, 0, 0, 0]
pull_request [0, 0, 3, 14, 16, 18, 8, 14, 13, 9, 6, 14, 11, 4, 7, 17, 15, 14, 13, 20, 24, 17, 7, 13, 8, 0, 0, 0]
issue [0, 0, 6, 24, 39, 29, 33, 39, 41, 33, 32, 50, 63, 26, 41, 52, 51, 47, 40, 39, 65, 54, 35, 39, 19, 0, 0, 0]

Quaterwise New Contributors Activity

commit [0, 78, 154, 94, 126, 33, 30, 38, 45, 37, 27, 80, 28, 29, 21, 34, 27, 28, 31, 57, 44, 26, 10, 37, 4, 0, 0, 0]
pull_request [0, 0, 3, 14, 16, 18, 8, 14, 13, 9, 6, 14, 11, 4, 7, 17, 15, 14, 13, 20, 24, 17, 7, 13, 8, 0, 0, 0]
issue [0, 0, 6, 24, 39, 29, 33, 39, 41, 33, 32, 50, 63, 26, 41, 52, 51, 47, 40, 39, 65, 54, 35, 39, 19, 0, 0, 0]


## Show the Output as a table

In [10]:
qinfo = pd.DataFrame()
qinfo['Quarter'] = quarters
qinfo['# Commits'] = activities['commit']
qinfo['# Issues'] = activities['issue']
qinfo['# PullRequests'] = activities['pull_request']
qinfo['# NewCommitters'] = newcontributors['commit']
qinfo['# NewIssueSubmitters'] = newcontributors['issue']
qinfo['# NewPRSubmitters'] =  newcontributors['pull_request']

In [11]:
qinfo.to_csv('elasticsearch-py.csv',index=None)

In [12]:
pd.read_csv('elasticsearch-py.csv')

,Quarter,# Commits,# Issues,# PullRequests,# NewCommitters,# NewIssueSubmitters,# NewPRSubmitters
0,Q1 2013,0,0,0,0,0,0
1,Q2 2013,78,0,0,1,0,0
2,Q3 2013,154,6,3,2,5,2
3,Q4 2013,94,24,14,9,16,9
4,Q1 2014,126,39,16,6,32,16
5,Q2 2014,33,29,18,11,25,15
6,Q3 2014,30,33,8,3,25,8
7,Q4 2014,38,39,14,3,30,11
8,Q1 2015,45,41,13,8,28,12
9,Q2 2015,37,33,9,4,31,7
